This notebook aims to calibrate a camera using a checkerboard pattern and image processing techniques. Here's a summary of the workflow:

1. **Checkerboard Generation**:
   - A checkerboard pattern is generated with known 3D coordinates. Each square of the checkerboard represents a known physical distance. The coordinates are defined with one dimension representing the Z-axis (set as 1 for simplicity), while the other two dimensions represent the X and Y coordinates.

2. **Image Capture**:
   - An image of the checkerboard is captured using the camera to be calibrated. This image serves as the input for camera calibration.

3. **Mouse Click Event**:
   - Using OpenCV, a mouse click event is enabled to allow the user to click on the corners of the squares in the checkerboard image. These clicks represent the destination points in the image corresponding to the known 3D coordinates.

4. **Projection Matrix Calculation**:
   - The `get_projection_matrix` function calculates the entries of the Homography matrix between the known 3D coordinates and the clicked destination points in the image. This matrix transformation represents the mapping between the world coordinates and the image coordinates.

5. **Camera Calibration**:
   - The projection matrix is computed using Singular Value Decomposition (SVD) techniques, which effectively captures the transformation from the checkerboard's 3D space to the 2D image plane. This matrix is crucial for camera calibration.

6. **Conclusion**:
   - The notebook concludes with the display of the computed projection matrix, which serves as a fundamental component in camera calibration. By relating known 3D coordinates to their corresponding image locations, the camera's intrinsic and extrinsic parameters can be accurately estimated, facilitating tasks such as image rectification, 3D reconstruction, and pose estimation.

Overall, the notebook provides a systematic approach to camera calibration using a checkerboard pattern and interactive click handling, enabling accurate mapping between 3D world coordinates and 2D image coordinates.

In [1]:
import cv2
import numpy as np

In [2]:
def click_event(event, x, y, flags, params):
    # Check for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        # Append the coordinates of the click to the clicks list
        clicks.append([x, y])

        # Display the coordinates on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(params, f"{x},{y}", (x, y), font, 1, (255, 0, 0), 2)
        cv2.imshow('image', params)

    # Check for right mouse clicks
    if event == cv2.EVENT_RBUTTONDOWN:
        # Append the coordinates of the click to the clicks list
        clicks.append([x, y])

        # Display the coordinates and color values on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        b, g, r = params[y, x]  # Extract BGR values from the clicked pixel
        cv2.putText(params, f"{b},{g},{r}", (x, y), font, 1, (255, 255, 0), 2)
        cv2.imshow('image', params)

This method `click_event` is designed to handle mouse click events on an image using OpenCV (assuming `cv2` refers to OpenCV). Let's break down its functionality:

1. **Function Parameters**: 
   - `event`: Represents the type of mouse event (e.g., left button down, right button down).
   - `x` and `y`: Represent the coordinates of the mouse click.
   - `flags`: Additional flags provided by OpenCV (not used in this method).
   - `params`: Additional parameters that can be passed to the function.

2. **Handling Left Mouse Clicks**:
   - When a left mouse button is clicked (`cv2.EVENT_LBUTTONDOWN`), the coordinates `(x, y)` of the click are appended to a list called `clicks`.
   - The coordinates of the click are then displayed on the image window using `cv2.putText()` function with the position `(x, y)` on the image.
   - The updated image with the displayed coordinates is shown using `cv2.imshow()`.

3. **Handling Right Mouse Clicks**:
   - Similarly, when a right mouse button is clicked (`cv2.EVENT_RBUTTONDOWN`), the coordinates `(x, y)` of the click are appended to the `clicks` list.
   - Additionally, the color values (BGR) of the pixel at the clicked coordinates are extracted using `params[y, x]`.
   - The extracted color values along with the coordinates are displayed on the image window using `cv2.putText()`.
   - The updated image with the displayed color values and coordinates is shown using `cv2.imshow()`.

Overall, this method allows for interactive exploration of an image by displaying the coordinates of left mouse clicks and the color values of right mouse clicks.

In [3]:
# Initialize the list to store click coordinates
clicks = []

# Read the image
two_d = cv2.imread("two_d.jpg")

# Set the image as the parameter for the click event function
param = two_d

# Display the image and set mouse callback function
cv2.imshow('image', two_d)
cv2.setMouseCallback('image', click_event, param)

# Wait for a key to be pressed to exit
cv2.waitKey(0)

# Close the window
cv2.destroyAllWindows()

# Store the coordinates of clicks in a variable
dest_clicks = clicks

This script utilizes the `click_event` method to enable interactive click handling on an image using OpenCV. Let's break down what each part does:

1. **Initializing Clicks List**: 
   - `clicks = []`: This initializes an empty list called `clicks` which will store the coordinates of mouse clicks.

2. **Reading the Image**:
   - `two_d = cv2.imread("two_d.jpg")`: Reads an image named "two_d.jpg" from the current directory (assuming it's present) using OpenCV's `imread` function. This image will be used for mouse click handling.

3. **Setting Parameters**:
   - `param = two_d`: Sets the variable `param` to the image `two_d`, which will be passed as a parameter to the `click_event` function.

4. **Displaying the Image**:
   - `cv2.imshow('image', two_d)`: Displays the image window with the title "image" containing the loaded image `two_d`.
   - `cv2.setMouseCallback('image', click_event, param)`: Sets a mouse callback function `click_event` on the image window. This function will handle mouse events on the displayed image. The `param` variable (image) is passed as a parameter to the callback function.

5. **Waiting for User Input**:
   - `cv2.waitKey(0)`: Waits indefinitely for a key press. The argument `0` indicates that the program will wait until any key is pressed.
   
6. **Closing Windows**:
   - `cv2.destroyAllWindows()`: Closes all OpenCV windows.

7. **Storing Click Coordinates**:
   - `dest_clicks = clicks`: Stores the coordinates of clicks in the `dest_clicks` variable for further processing.

This script allows the user to interactively click on the displayed image, capturing the coordinates of each click. After the user finishes clicking and presses any key, the program exits, and the coordinates are stored in the `dest_clicks` variable for later use.

In [13]:
source_clicks = [[1,3,2],[1,5,3],[1,6,4],[1,3,6],[1,6,7],[1,8,2]]
source_clicks 

[[1, 3, 2], [1, 5, 3], [1, 6, 4], [1, 3, 6], [1, 6, 7], [1, 8, 2]]

In [14]:
dest_clicks

[[411, 370], [287, 304], [223, 241], [404, 119], [220, 61], [86, 371]]

In [15]:
import numpy as np

def get_projection_matrix(source, destination):
    """Calculates the entries of the Homography matrix between two sets of matching points.

    Args:
        source (list): Source points where each point is in (x, y, z) format.
        destination (list): Destination points where each point is in (x, y) format.

    Returns:
        numpy.ndarray: A numpy array of shape (4, 3) representing the projection matrix.

    Raises:
        AssertionError: If source or destination has less than six points or if they are of different sizes.
    """
    assert len(source) >= 6, "Must provide more than 6 source points"
    assert len(destination) >= 6, "Must provide more than 6 destination points"
    assert len(source) == len(destination), "Source and destination must be of equal length"

    A = []
    for i in range(len(source)):
        s_x, s_y, s_z = source[i]
        d_x, d_y = destination[i]

        # Constructing the matrix A
        A.append([s_x, s_y, s_z, 1, 0, 0, 0, 0, -d_x * s_x, -d_x * s_y, -d_x * s_z, -d_x])
        A.append([0, 0, 0, 0, s_x, s_y, s_z, 1, -d_y * s_x, -d_y * s_y, -d_y * s_z, -d_y])

    # Convert A to a numpy array
    A = np.array(A)
    return A

This function, `get_projection_matrix`, calculates the entries of the Homography matrix between two sets of matching points. Here's a breakdown of its functionality:

1. **Function Parameters**:
   - `source`: List of source points where each point is represented as (x, y, z).
   - `destination`: List of destination points where each point is represented as (x, y).

2. **Input Validation**:
   - Asserts that the length of `source` and `destination` lists is at least 6 (minimum required points for Homography calculation).
   - Ensures that the lengths of `source` and `destination` lists are equal.

3. **Constructing Matrix A**:
   - Initializes an empty list `A` to store the entries of the matrix A.
   - Iterates over each pair of corresponding points in `source` and `destination`.
   - For each pair, extracts the coordinates `(s_x, s_y, s_z)` from `source` and `(d_x, d_y)` from `destination`.
   - Constructs two rows of the matrix A based on the equations of Homography.
   - Appends these rows to the list `A`.

4. **Conversion to Numpy Array**:
   - Converts the list `A` to a numpy array for further computation.

5. **Returns**:
   - Returns the numpy array `A`, representing the projection matrix.

This function effectively prepares the data for calculating the Homography matrix, a fundamental transformation used for perspective correction and image registration.

In [21]:
# Calculate the projection matrix A
A = get_projection_matrix(source_clicks, dest_clicks)

# Calculate A transpose multiplied by A
projection_matrix = A.T @ A

# Perform Singular Value Decomposition (SVD)
u, D, v = np.linalg.svd(projection_matrix, full_matrices=True)

# Extract the last row of v and reshape it into a 3x4 projection matrix
projection_matrix = v[-1].reshape(3, 4)

# Display the projection matrix
projection_matrix

array([[-7.06123299e-01, -8.27134992e-10,  4.66652674e-11,
         7.06123307e-01],
       [-3.72505796e-02, -1.82721343e-11, -7.58523375e-10,
         3.72505858e-02],
       [ 1.50834160e-03, -7.03113782e-14,  1.90143935e-13,
        -1.50834159e-03]])

This script utilizes the projection matrix calculation based on the provided source and destination points. Here's a breakdown of each step:

1. **Calculation of Projection Matrix A**:
   - `A = get_projection_matrix(source_clicks, dest_clicks)`: Calls the `get_projection_matrix` function with `source_clicks` and `dest_clicks`, which are the lists of source and destination points respectively, to calculate the projection matrix.

2. **Calculation of A Transpose multiplied by A**:
   - `projection_matrix = A.T @ A`: Calculates the product of the transpose of matrix A and A itself.

3. **Singular Value Decomposition (SVD)**:
   - `u, D, v = np.linalg.svd(projection_matrix, full_matrices=True)`: Performs Singular Value Decomposition (SVD) on the `projection_matrix` to obtain matrices `u`, `D`, and `v`.

4. **Extracting Projection Matrix from SVD**:
   - Extracts the last row of matrix `v`, which corresponds to the singular vector corresponding to the smallest singular value.
   - Reshapes this vector into a 3x4 projection matrix.

5. **Displaying the Projection Matrix**:
   - The calculated projection matrix is displayed.

This script effectively computes the projection matrix required for perspective transformation or image rectification, based on the provided source and destination points.